<a href="https://colab.research.google.com/github/nadiarichards/ML-in-Google-cloud/blob/main/EmotionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd

/root


In [ ]:
!pwd

/root


In [ ]:
!pip install h5py

In [ ]:
import numpy as np
import os
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
# from keras.emotion_models import Sequential
# from tensorflow.keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
# drive.mount('/content/gdrive')
# /content/gdrive/My Drive/Machine Learning Images Folder/train_test_images/train

train_dir = "/content/drive/My Drive/Machine Learning Images Folder/train_test_images/train"
val_dir = "/content/drive/My Drive/Machine Learning Images Folder/train_test_images/test"
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
# emotion_model.load_weights('emotion_model.h5')

cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)
emotion_model.save('/content/drive/My Drive/Machine Learning Images Folder/emotion_model.h5')

Found 28711 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
448/448 [==============================] - 8425s 19s/step - loss: 1.8295 - accuracy: 0.2432 - val_loss: 1.7000 - val_accuracy: 0.3598
Epoch 2/50
448/448 [==============================] - 64s 143ms/step - loss: 1.6482 - accuracy: 0.3591 - val_loss: 1.5307 - val_accuracy: 0.4213
Epoch 3/50
448/448 [==============================] - 64s 144ms/step - loss: 1.5374 - accuracy: 0.4087 - val_loss: 1.4601 - val_accuracy: 0.4460
Epoch 4/50
448/448 [==============================] - 64s 143ms/step - loss: 1.4607 - accuracy: 0.4434 - val_loss: 1.4146 - val_accuracy: 0.4605
Epoch 5/50
448/448 [==============================] - 64s 143ms/step - loss: 1.4113 - accuracy: 0.4624 - val_loss: 1.3799 - val_accuracy: 0.4752
Epoch 6/50
448/448 [==============================] - 63s 142ms/step - loss: 1.3603 - accuracy: 0.4807 - val_loss: 1.3243 - val_accuracy: 0.4975
Epoch 7/50
448/448 [==============================] - 64s 142ms/step - loss: 1.3184 - accuracy: 0.4996 - val_loss: 1.2845 - val_ac

In [ ]:
import numpy as np
import os
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
# from keras.emotion_models import Sequential
# from tensorflow.keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive

In [ ]:
# from tensorflow.keras.models import load_weights
emotion_model = load_weights("/content/drive/My Drive/Machine Learning Images Folder/emotion_model.h5")

NameError: ignored

In [ ]:
# start the webcam feed
emotion_model.load_weights('emotion_model.h5')
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('/home/shivam/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2gray_frame)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# emotion_model.load_weights('emotion_model.h5')
model_save_name = 'emotions.h5'
path = F"/content/gdrive/My Drive/emotions.h5" 
emotion_model.save_weights('emotions.h5')

In [ ]:
from keras.models import model_from_json

In [ ]:
model_json = emotion_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
emotion_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_yaml

In [ ]:
model_yaml = emotion_model.to_yaml()
with open("emotion_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
emotion_model.save_weights("emotion_model_yaml.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
emotion_model.load_weights('emotion_model.h5')

NameError: ignored